In [ ]:
import numpy as np
import socket
import time

In [ ]:
# Vocabolario di azioni “fisso” che Unity si aspetta (esempio one-hot a 5 lunghezze)
ACTIONS = {
    'LEFT':     [1, 0, 0, 0, 0, 0], # indice 0 => mano sinistra
    'RIGHT':    [0, 1, 0, 0, 0, 0], # indice 1 => mano destra
    'FORWARD':  [0, 0, 1, 0, 0, 0], # indice 2 => piedi
    'LINGUA':   [0, 0, 0, 1, 0, 0], # indice 3 => lingua
    'STOP':     [0, 0, 0, 0, 1, 0], # indice 4 => stop (rest)
    'PIEDE':    [0, 0, 0, 0, 0, 1], # indice 5 => piede
}

# Mappature “dataset → azione” basate su come interpreti le classi interne
DATASET_MAPPINGS = {
    'BCI2a': {
        0: ACTIONS['LEFT'],    # indice 0 => mano sinistra => LEFT
        1: ACTIONS['RIGHT'],   # indice 1 => mano destra => RIGHT
        2: ACTIONS['FORWARD'], # indice 2 => piedi => FORWARD
        3: ACTIONS['LINGUA'],    # indice 3 => lingua => Lingua
        'default': ACTIONS['STOP']
    },
    'BCI2b': {
        0: ACTIONS['LEFT'],    # indice 0 => mano sinistra => LEFT
        1: ACTIONS['RIGHT'],   # indice 1 => mano destra => RIGHT
        'default': ACTIONS['STOP']
    },
    'HGD': {
        0: ACTIONS['RIGHT'],   # indice 0 => mano destra => RIGHT
        1: ACTIONS['LEFT'],    # indice 1 => mano sinistra => LEFT
        2: ACTIONS['STOP'],    # indice 2 => rest => STOP
        3: ACTIONS['PIEDE'], # indice 3 => piede => Piede
        'default': ACTIONS['STOP']
        # … aggiungi tutte le classi che hai, mappandole su questo “vocabolario”
    }
}

def get_action_for(dataset_name, class_index):
    """
    Dato il nome del dataset (stringa) e l'indice di classe interno,
    restituisce l'array one-hot di lunghezza fissa (5 nel nostro esempio)
    che Unity sa interpretare.
    """
    mapping = DATASET_MAPPINGS[dataset_name]
    return mapping.get(class_index, mapping['default'])


In [ ]:
import os
import numpy as np

def load_preds_and_trues(base_folder, dataset_name, subject_id):
    """
    Carica preds e true per un dato dataset e un dato soggetto,
    assumendo che la struttura delle cartelle sia:
    
      base_folder/
        ├─ BCI2a/
        │   ├─ BCI2a_sub1_cmds.npy
        │   ├─ BCI2a_sub1_true.npy
        │   ├─ BCI2a_sub2_cmds.npy
        │   ├─ BCI2a_sub2_true.npy
        │   └─ ...
        ├─ BCI2b/
        │   ├─ BCI2b_sub1_cmds.npy
        │   ├─ BCI2b_sub1_true.npy
        │   └─ ...
        └─ HGD/
            ├─ HGD_sub1_cmds.npy
            ├─ HGD_sub1_true.npy
            └─ ...
    
    Parametri:
      - base_folder: cartella “comandi” che contiene le sottocartelle dei dataset
                     (es. "commands" o "./commands")
      - dataset_name: nome del dataset ("BCI2a", "BCI2b" o "HGD")
      - subject_id:   numero del soggetto (intero o stringa) senza gli zeri iniziali,
                      ad esempio "1", "2", … fino a "9" o "14" per HGD.
    
    Ritorna:
      - preds: numpy array con shape (n_trials,) o (n_trials, n_classi)
      - trues: numpy array con shape (n_trials,) o (n_trials, n_classi)
    """

    # 1) Controllo che il dataset sia supportato
    if dataset_name not in {"BCI2a", "BCI2b", "HGD"}:
        raise ValueError(
            f"Dataset '{dataset_name}' non supportato. Scegli tra 'BCI2a', 'BCI2b' o 'HGD'."
        )

    # 2) Cartella specifica del dataset
    dataset_folder = os.path.join(base_folder, dataset_name)
    if not os.path.isdir(dataset_folder):
        raise FileNotFoundError(
            f"La cartella del dataset non esiste:\n  {dataset_folder}"
        )

    # 3) Costruisco i nomi dei file secondo la convenzione:
    #    <dataset_name>_sub<subject_id>_cmds.npy
    #    <dataset_name>_sub<subject_id>_true.npy
    pred_filename = f"{dataset_name}_sub{subject_id}_cmds.npy"
    true_filename = f"{dataset_name}_sub{subject_id}_true.npy"

    pred_path = os.path.join(dataset_folder, pred_filename)
    true_path = os.path.join(dataset_folder, true_filename)

    # 4) Controllo esistenza fisica dei file
    if not os.path.isfile(pred_path):
        raise FileNotFoundError(
            f"File di predizioni non trovato:\n  {pred_path}"
        )
    if not os.path.isfile(true_path):
        raise FileNotFoundError(
            f"File di true_label non trovato:\n  {true_path}"
        )

    # 5) Carico i .npy in numpy array
    preds = np.load(pred_path, allow_pickle=True)
    trues = np.load(true_path, allow_pickle=True)

    # 6) Verifico consistenza di lunghezza
    if preds.shape[0] != trues.shape[0]:
        raise ValueError(
            f"I due file hanno numeri di trial diversi:\n"
            f"  preds ha {preds.shape[0]} righe, trues ha {trues.shape[0]} righe."
        )

    return preds, trues


In [ ]:
# =========================================================
# Imposta questi parametri in base alla tua struttura
# =========================================================
BASE_FOLDER  = "commands"      # cartella che contiene le sottocartelle BCI2a, BCI2b e HGD
DATASET_NAME = "BCI2a"         # o "BCI2b" o "HGD"
SUBJECT_ID   = "3"             # ad es. "1", "2", … (per BCI2a/BCI2b vanno da 1 a 9; HGD da 1 a 14)
RATE_HZ      = 2.0             # velocità di invio comandi (es. 2 comandi al secondo)

# Caricamento predizioni e true‐label
preds, trues = load_preds_and_trues(BASE_FOLDER, DATASET_NAME, SUBJECT_ID)
print(f"Dataset {DATASET_NAME}, Subject {SUBJECT_ID}, Numero di trial: {len(preds)}")


In [ ]:
# Parametri di connessione - Unity deve essere in ascolto sulla stessa porta
HOST = "127.0.0.1"
PORT = 5005

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))
print("Connesso a Unity su", HOST, "porta", PORT)

In [ ]:
# 3) Inizializzo fallback e contatori
n_trials = preds.shape[0]
delay    = 1.0 / RATE_HZ
correct  = 0

# Primo fallback col primo true-label
first_true = trues[0]
if isinstance(first_true, (list, np.ndarray)) and len(np.asarray(first_true)) > 1:
    idx0_true = int(np.argmax(first_true))
else:
    idx0_true = int(first_true)
last_cmd = get_action_for(DATASET_NAME, idx0_true)

# 4) Loop invio
for i in range(n_trials):
    p = preds[i]
    t = trues[i]

    # 4.1) Estrazione indici
    if isinstance(p, (list, np.ndarray)) and len(np.asarray(p)) > 1:
        idx_pred = int(np.argmax(p))
    else:
        idx_pred = int(p)

    if isinstance(t, (list, np.ndarray)) and len(np.asarray(t)) > 1:
        idx_true = int(np.argmax(t))
    else:
        idx_true = int(t)

    # 4.2) Mappatura in azioni
    cmd_pred = get_action_for(DATASET_NAME, idx_pred)
    cmd_true = get_action_for(DATASET_NAME, idx_true)

    # 4.3) Controllo correttezza
    if cmd_pred == cmd_true:
        to_send = cmd_pred
        correct += 1
        note = "ESEGUO"
        last_cmd = cmd_true
    else:
        to_send = last_cmd
        note = f"ERRATO → mantengo {last_cmd}"
        # da modificare se bisogna un fallback diverso
        # (es. un'azione di ripristino o un default specifico)
        # per ora mantengo l'ultima azione corretta

    # 4.4) Serializzo e invio a Unity
    msg = ",".join(str(x) for x in to_send) + "\n"
    sock.send(msg.encode())

    # 4.5) Log
    print(f"Trial {i:03d}: pred_idx={idx_pred}->{cmd_pred}, true_idx={idx_true}->{cmd_true} | {note}")

    # 4.6) Attesa
    time.sleep(delay)

# 5) Accuracy finale
accuracy = correct / n_trials
print(f"\nAccuracy complessiva = {accuracy:.2%} ({correct}/{n_trials})")

sock.close()